# Sample Code

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
import os 
print(os.getcwd())
metadata = getDF('C:/Users/User/PycharmProjects/data-course-sample/meta_All_Beauty.json.gz')
ratings = pd.read_csv('C:/Users/User/PycharmProjects/data-course-sample/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

C:\Users\User\PycharmProjects\data-course-sample\week_2


In [3]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [4]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [5]:
import regex as re
from string import digits

def remove_brand(text):
    res = re.sub(brand_pattern, '', text)
    return res

def find_brand(text):
    res = re.match(brand_pattern, text)
    return res.group(0) if res else ''

def extract_price(cell):
    cell = ''.join(c for c in cell if c in digits)
    return int(cell) if len(cell) > 0 else 0
    
def find_sub_cat(cell):
    if len(cell) > 0:
        cell = str(cell).split('in ')
        cell = re.sub(r"((&amp)|&|;|'|(100)|Top|top|See| )*", '', cell[1])
        return cell.replace("\(", '').replace('(', '').replace(')', '').replace(']', '')
    else:
        return 'without_category'

ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
brand_list = metadata['brand'].value_counts().keys().tolist()
brand_list = list(filter(lambda x:len(x)>2, brand_list))
brand_pattern = re.compile(r"(?:{})".format('|'.join([re.escape(x) for x in brand_list])))
metadata['brand_from_title'] = metadata['title'].apply(find_brand)
metadata['title_without_brand'] = metadata['title'].apply(remove_brand)
metadata['price'] = metadata['price'].apply(extract_price)
metadata['sub_category'] = metadata['rank'].apply(find_sub_cat)

In [6]:
metadata['sub_category'].value_counts()

BeautyPersonalCare       32442
without_category           377
GroceryGourmetFood          39
HealthHousehold             21
SportsOutdoors               3
ToysGames                    3
Baby                         2
Clothing,ShoesJewelry        2
HomeKitchen                  1
Automotive                   1
ToolsHomeImprovement         1
Name: sub_category, dtype: int64

In [7]:
metadata = metadata[(metadata['sub_category']=='BeautyPersonalCare')]
metadata

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,...,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,brand_from_title,title_without_brand,sub_category
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",...,All Beauty,,,0,6546546450,[],[],,Loud 'N Clear&trade; Personal Sound Amplifier,BeautyPersonalCare
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",...,All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,4499,7178680776,[],[],No7,Lift &amp; Luminate Triple Action Serum 50ml by,BeautyPersonalCare
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (",...,All Beauty,,,2876,7250468162,[],[],No7,Stay Perfect Foundation Cool Vanilla by,BeautyPersonalCare
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",...,All Beauty,,,0,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,Wella,Koleston Perfect Hair Colour 44/44 um Intense...,BeautyPersonalCare
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (",...,All Beauty,,,1215,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,,Lacto mine Skin Balance Oil control 120 ml. (P...,BeautyPersonalCare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32887,[],,[],,"Barielle Pro Textured Grip Cuticle Nipper, Purple",[],,,[],"2,145,325 in Beauty & Personal Care (",...,All Beauty,,,995,B01HIWLLUK,[],[],,"Barielle Pro Textured Grip Cuticle Nipper, Purple",BeautyPersonalCare
32888,[],,[],,(Buy 3 Get 1 Free) Salon Perfect Eye Makeup Co...,[],,Salon Perfect,[],"1,639,713 in Beauty & Personal Care (",...,All Beauty,,,0,B01HJ1K3YK,[],[],,(Buy 3 Get 1 Free) Eye Makeup Corrector Stick...,BeautyPersonalCare
32889,[],,[],,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,"[B01KON9B4S, B079X3YFXS, B00M79OYS6, B000JN4CR...",,,[],"207,410 in Beauty & Personal Care (",...,All Beauty,,,5563,B01HJ84SGM,[],[],NOW,D-Mannose 500 mg - 120 Veg Capsules (Pack of 3),BeautyPersonalCare
32890,[],,[Brand new and high quality<br> Enables fast v...,,12 White Feather Shuttlecocks Birdies Badminto...,[],,GBSTORE,[],"965,673 in Beauty & Personal Care (",...,All Beauty,,,1299,B01HJASD20,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,,12 White Shuttlecocks Birdies Badminton Train...,BeautyPersonalCare


## 資料切分

In [8]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 處理文字資料

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
# 計算商品用標題所表示的 tfidf 矩陣
df = metadata.drop_duplicates('title')
title_without_brand = df['title_without_brand'].to_list()
nlp = spacy.load('en_core_web_sm', disable = ['ner', 'tagger', 'parser', 'textcat'])

for idx, sent in enumerate(title_without_brand):
    title_without_brand[idx] = re.sub('[0-9.,-_]*', '', sent)
title_without_brand = list(nlp.pipe(title_without_brand))

C:\Users\User\.conda\envs\spacy_env\lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [32]:
title_without_brand_feature = list()
for idx, sent in enumerate(title_without_brand):
    title = list()
    for word in target:
        if word.is_alpha:
            title.append(word.norm_)
    title_without_brand_feature.append(' '.join(title))
print(len(title_without_brand_feature))            

31892


In [33]:
vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_matrix = vectorizer.fit_transform(title_without_brand_feature)

In [34]:
# 計算商品間的相似程度
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(tfidf_matrix)
mapping = pd.Series(df.index,index = df['title'])

# 每個商品回傳 k 個最相近的商品
def recommend_item(item_input, k=2):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (df['asin'].iloc[item_indices].tolist())
    except:
        return []

# 利用使用者購買過的商品產生推薦
def recommend_items(items, k):
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    return res


In [51]:
print(df['title'][0:11].to_list())
print(recommend_items(df['title'][0:11].to_list(), k=10))

["Loud 'N Clear&trade; Personal Sound Amplifier", 'No7 Lift &amp; Luminate Triple Action Serum 50ml by Boots', 'No7 Stay Perfect Foundation Cool Vanilla by No7', 'Wella Koleston Perfect Hair Colour 44/44 Medium Intense Red Brown 60ml', 'Lacto Calamine Skin Balance Oil control 120 ml. (Pack of 2)', 'Mary Kay Satin Hands Hand Cream Travel MINI Size Set of 6', 'Unique Custom Cast Iron Liner Shader Tattoo Machine E010541', 'Imagen Bendita Por Su Santidad Our Lady of Charity Caridad Del Cobre Patrona De Cuba Cachita B-36 Small - 6 Inches', 'Mali princ', "Moleskine Payne's Grey Card Wallet (Moleskine Non-Paper)", "L'eternauta, il romanzo"]
['6546546450', '7178680776', '7250468162', '7367905066', '7414204790', '7535842801', '7538626107', '8279996397', '8637910351', '8867326759', '6546546450', '7178680776', '7250468162', '7367905066', '7414204790', '7535842801', '7538626107', '8279996397', '8637910351', '8867326759', '6546546450', '7178680776', '7250468162', '7367905066', '7414204790', '753584

In [ ]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    content-based
    '''
    ratings_trainings = training_data
    recommendations = {user: recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), k) for user in users}
    return recommendations

ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

## 結果評估

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)